In [2]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

## Load the training data into feature matrix, class labels, and event ids:

In [6]:
from proj1_helpers import *
DATA_TRAIN_PATH = 'data/train_unzip.csv' 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

## Data Visualization

## Data Cleanup

In [23]:
# Remove data with more than 70% missing values and store the output in x1 array
x1=tX
missing_percent = np.zeros(len(x1[0,:]))
new_missing_percent = []
i = 0
j = 0
while True:
    if(i==len(x1[0,:])):
            break
    a = (x1[:,i]==-999)+ np.zeros(len(x1[:,i]))
    c = np.sum(a)/len(a)
    missing_percent[j] = c
    j+=1
    if c>=0.7:
        x1= np.delete(x1,i,1)
        continue
    new_missing_percent = np.append(new_missing_percent,c)
    i+=1
    

print("Missing values' percentage for each column is \n",  missing_percent)
print("New missing values' percentage after 70% threshold \n",  new_missing_percent)
print("Intial number of features is \n", len(tX[0,:]))
print("Number of deleted features is \n", len(tX[0,:])-len(x1[0,:]))


Missing values' percentage for each column is 
 [0.152456 0.       0.       0.       0.709828 0.709828 0.709828 0.
 0.       0.       0.       0.       0.709828 0.       0.       0.
 0.       0.       0.       0.       0.       0.       0.       0.399652
 0.399652 0.399652 0.709828 0.709828 0.709828 0.      ]
New missing values' percentage after 70% threshold 
 [0.152456 0.       0.       0.       0.       0.       0.       0.
 0.       0.       0.       0.       0.       0.       0.       0.
 0.       0.       0.       0.399652 0.399652 0.399652 0.      ]
Intial number of features is 
 30
Number of deleted features is 
 7


In [24]:
# Data cleaning 2: For each feature, replace each missing value with mean of feature 
s = (len(x1[:,0]),len(x1[0,:]))
logic_array = np.ones(s)
for k in range(len(x1[0,:])):
    
    if(new_missing_percent[k]>0):
        logic_array[:,k] = (x1[:,k]!=-999)+ np.zeros(len(a))
x2= np.multiply(logic_array,x1) #array with zeros instead of -999

mean = np.mean(x2,0) #vector with mean of each column without -999
means = np.multiply(1-logic_array,mean) # zeros-array with mean of column instead of missing value
x3=np.add(x2,means) # array cleaned, each -999 is replaced with mean of column


    

## Do your thing crazy machine learning thing here :) ...

## Generate predictions and save ouput in csv format for submission:

In [10]:
DATA_TEST_PATH = '' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [31]:
OUTPUT_PATH = '' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)